# BP-UCB: Budget-Feasible Posted-Price UCB

This notebook demonstrates the BP-UCB algorithm from:

> Singla, A. and Krause, A. (2013). **Truthful Incentives in Crowdsourcing Tasks using Regret Minimization Mechanisms.** WWW '13.

The algorithm learns optimal posted prices for hiring workers under budget constraints using Upper Confidence Bounds.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../src")

from ucb_pricing import BPUCB

# Enable LaTeX rendering
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif')

## Problem Setup

A platform wants to hire workers under a budget constraint:
- Budget $B = 1000$
- Worker pool $N = 12000$
- Worker costs drawn from a distribution on $[c_{min}, c_{max}]$
- $K = 11$ discretized price levels

The optimal price $p^*$ is where $N \cdot F(p) = \dfrac{B}{p}$.

In [ ]:
# Parameters (matching the original paper/notebook)
B = 1000
N = 12000
c_min = 0.001
c_max = 1.0
K = 11

def generate_bids(n, c_min, c_max):
    """Generate worker bids with noise to create realistic cost distribution."""
    return np.array([
        min(max(random.uniform(c_min, c_max) + random.uniform(-0.3, 0.3), c_min), c_max)
        for _ in range(n)
    ])

# Initialize algorithm with linear price spacing
bp = BPUCB(budget=B, n_workers=N, c_min=0.0, c_max=c_max, n_prices=K, alpha=None)
bp.prices[0] = 0.01  # Avoid zero price

# Generate worker bids
random.seed(42)
bids = generate_bids(N, c_min, c_max)
bp.set_bids(bids)

print(f"Prices: {bp.prices}")
print(f"Optimal discretized price (p'): {bp.p_prime:.4f}")
print(f"Optimal continuous price (p*): {bp.p_star:.4f}")

## Cost Curve and Budget Constraint

The optimal price lies at the intersection of:
- $N \cdot F(p)$: Expected workers accepting price $p$
- $B / p$: Maximum workers affordable at price $p$

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(bp.prices, N * bp.F, 'g-o', label=r'$N \cdot F(p)$')
ax.plot(bp.prices, B / bp.prices, 'b-o', label=r'$\frac{B}{p}$')
ax.axvline(bp.p_star, color='red', linestyle='--', label=f'$p^* = {bp.p_star:.3f}$')
ax.axvline(bp.p_prime, color='purple', linestyle='--', label=f"$p' = {bp.p_prime:.3f}$")

ax.set_xlabel('Price')
ax.set_ylabel('Expected Number of Workers')
ax.set_title('Cost Curve and Budget Constraint')
ax.legend()
ax.set_ylim(0, 12000)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../assets/figure1_original.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved to assets/figure1_original.png")

## Running BP-UCB

In [ ]:
result = bp.run()

print(f"BP-UCB Utility: {result.utility}")
print(f"Optimal Utility (B/p*): {B / bp.p_star:.0f}")
print(f"Regret: {bp.compute_regret(result):.0f}")
print(f"Efficiency: {result.utility / (B / bp.p_star) * 100:.1f}%")

## Convergence Analysis

Run multiple simulations to see how often the algorithm selects the optimal price.

In [ ]:
n_simulations = 100
mean_rates = np.zeros(N)

for sim in range(n_simulations):
    if sim % 20 == 0:
        print(f"Simulation {sim}/{n_simulations}")
    
    bids = generate_bids(N, c_min, c_max)
    bp_sim = BPUCB(budget=B, n_workers=N, c_min=0.0, c_max=c_max, n_prices=K, alpha=None)
    bp_sim.prices[0] = 0.01
    bp_sim.set_bids(bids)
    
    result = bp_sim.run()
    i_prime = bp_sim.i_prime
    
    for t, idx in enumerate(result.price_indices):
        mean_rates[t] += (idx == i_prime) / n_simulations

print("Done!")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(range(K, N), mean_rates[K:N])
ax.set_xlabel('Iteration')
ax.set_ylabel(r"Rate of Choosing $p'$")
ax.set_title(r"Convergence to Optimal Price Selection")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../assets/convergence.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved to assets/convergence.png")

## Utility vs Budget

Analyze how utility scales with different budget levels.

In [ ]:
budgets = list(range(100, 2400, 100))
avg_utilities = []
avg_optimal = []
n_simulations = 20

for budget in budgets:
    utils = []
    opts = []
    for _ in range(n_simulations):
        bp_sim = BPUCB(budget=budget, n_workers=N, c_min=0.0, c_max=c_max, n_prices=K, alpha=None)
        bp_sim.prices[0] = 0.01
        bids = generate_bids(N, c_min, c_max)
        bp_sim.set_bids(bids)
        result = bp_sim.run()
        utils.append(result.utility)
        opts.append(budget / bp_sim.p_star)
    avg_utilities.append(np.mean(utils))
    avg_optimal.append(np.mean(opts))

print("Done!")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(budgets, avg_optimal, 'b-^', label=r'Optimal $\frac{B}{p^*}$')
ax.plot(budgets, avg_utilities, 'g-o', label='BP-UCB')
ax.set_xlabel('Budget')
ax.set_ylabel('Average Utility')
ax.set_title('Utility vs Budget')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../assets/utility_vs_budget.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved to assets/utility_vs_budget.png")

## Price Selection Distribution

Analyze which prices the algorithm selects most often across multiple simulations.

In [ ]:
# Aggregate arm selection counts across simulations
n_simulations = 50
total_counts = np.zeros(K)

for sim in range(n_simulations):
    if sim % 10 == 0:
        print(f"Simulation {sim}/{n_simulations}")
    
    bp_sim = BPUCB(budget=B, n_workers=N, c_min=0.0, c_max=c_max, n_prices=K, alpha=None)
    bp_sim.prices[0] = 0.01
    bids = generate_bids(N, c_min, c_max)
    bp_sim.set_bids(bids)
    result = bp_sim.run()
    total_counts += result.counts

avg_counts = total_counts / n_simulations
print("Done!")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Highlight the optimal price in green
colors = ['green' if i == bp.i_prime else 'steelblue' for i in range(K)]
ax.bar(bp.prices, avg_counts, width=0.07, color=colors, alpha=0.8)

ax.axvline(bp.p_prime, color='red', linestyle='--', alpha=0.7, label=f"$p' = {bp.p_prime:.2f}$")
ax.set_xlabel(r"Price ($p$)")
ax.set_ylabel("Average Selection Count")
ax.set_title("Price Selection Distribution")
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../assets/price_selection.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved to assets/price_selection.png")

## Regret Analysis

Analyze how regret scales with budget (demonstrates no-regret property).

In [ ]:
# Compute regret across different budgets
budgets = list(range(100, 2400, 100))
avg_regrets = []
n_simulations = 20

for budget in budgets:
    regrets = []
    for _ in range(n_simulations):
        bp_sim = BPUCB(budget=budget, n_workers=N, c_min=0.0, c_max=c_max, n_prices=K, alpha=None)
        bp_sim.prices[0] = 0.01
        bids = generate_bids(N, c_min, c_max)
        bp_sim.set_bids(bids)
        result = bp_sim.run()
        regrets.append(bp_sim.compute_regret(result))
    avg_regrets.append(np.mean(regrets))

print("Done!")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Absolute regret
ax1.plot(budgets, avg_regrets, 'g-o')
ax1.set_xlabel('Budget')
ax1.set_ylabel('Average Regret')
ax1.set_title('Average Regret vs Budget')
ax1.grid(True, alpha=0.3)

# Normalized regret (regret / budget)
normalized_regrets = [r / b for r, b in zip(avg_regrets, budgets)]
ax2.plot(budgets, normalized_regrets, 'g-o')
ax2.set_xlabel('Budget')
ax2.set_ylabel('Average Regret / Budget')
ax2.set_title('Normalized Regret vs Budget')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../assets/regret_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved to assets/regret_analysis.png")